# Предсказание 

## Загрузка библиотек

In [ ]:
#import sys
#import os
#import warnings
#os.environ['OPENBLAS_NUM_THREADS'] = '1'


In [ ]:
#pip install pytorch-lifestream --user

In [ ]:
#pip install catboost

In [1]:
import pandas as pd
import numpy as np
import gc
import time
import pyarrow.parquet as pq
import scipy
#import implicit
import bisect
#import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import roc_auc_score, classification_report
from collections import Counter  
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [3]:
import torch
from ptls.data_load.datasets import ParquetDataset, parquet_file_scan
from ptls.preprocessing import PandasDataPreprocessor

In [4]:
import time
import datetime
import os

import pickle

In [5]:
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

## Обработка данных чтобы влезали в память

In [7]:
df_targ_1 = pd.read_pickle('C:\\data_sber_com\\processed_train.pickle\\processed_train_10.pickle')
df_targ_1.head(2)

,client_id_num,amount,event_time,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
0,0,"[tensor(11584946, dtype=torch.int32), tensor(14337, dtype=torch.int32), tensor(5506, dtype=torch.int32), tensor(107873, dtype=torch.int32), tensor(51, dtype=torch.int32), tensor(54826, dtype=torch.int32), tensor(10713, dtype=torch.int32), tensor(10022, dtype=torch.int32), tensor(5026, dtype=torch.int32), tensor(3, dtype=torch.int32), tensor(76889, dtype=torch.int32), tensor(456, dtype=torch.int32), tensor(5489, dtype=torch.int32), tensor(6965, dtype=torch.int32), tensor(770, dtype=torch.int32), tensor(6920812, dtype=torch.int32), tensor(8319, dtype=torch.int32), tensor(56051, dtype=torch.int32), tensor(4178, dtype=torch.int32), tensor(13929, dtype=torch.int32), tensor(6343, dtype=torch.int32), tensor(32819, dtype=torch.int32), tensor(1225, dtype=torch.int32), tensor(933, dtype=torch.int32), tensor(13423, dtype=torch.int32), tensor(17758, dtype=torch.int32), tensor(5261698, dtype=torch.int32), tensor(122, dtype=torch.int32), tensor(30696, dtype=torch.int32), tensor(882, dtype=torch.int32), tensor(7234, dtype=torch.int32), tensor(166, dtype=torch.int32), tensor(5260, dtype=torch.int32), tensor(3854, dtype=torch.int32), tensor(1792, dtype=torch.int32), tensor(906315, dtype=torch.int32), tensor(120, dtype=torch.int32), tensor(15511, dtype=torch.int32), tensor(25802, dtype=torch.int32), tensor(3419, dtype=torch.int32), tensor(426, dtype=torch.int32), tensor(1218, dtype=torch.int32), tensor(13725419, dtype=torch.int32), tensor(4887, dtype=torch.int32), tensor(8960170, dtype=torch.int32), tensor(155428, dtype=torch.int32), tensor(0, dtype=torch.int32), tensor(4424, dtype=torch.int32), tensor(8789, dtype=torch.int32), tensor(843, dtype=torch.int32), tensor(137740, dtype=torch.int32), tensor(92, dtype=torch.int32), tensor(1027, dtype=torch.int32), tensor(565, dtype=torch.int32), tensor(92, dtype=torch.int32), tensor(316354, dtype=torch.int32), tensor(1777, dtype=torch.int32), tensor(1, dtype=torch.int32), tensor(24, dtype=torch.int32), tensor(12852, dtype=torch.int32), tensor(3799, dtype=torch.int32), tensor(395, dtype=torch.int32), tensor(5445, dtype=torch.int32), tensor(8338, dtype=torch.int32), tensor(290, dtype=torch.int32), tensor(40652, dtype=torch.int32), tensor(16612, dtype=torch.int32), tensor(211773, dtype=torch.int32), tensor(36, dtype=torch.int32), tensor(3, dtype=torch.int32), tensor(2873, dtype=torch.int32), tensor(88664, dtype=torch.int32), tensor(37692, dtype=torch.int32), tensor(61, dtype=torch.int32), tensor(133, dtype=torch.int32), tensor(499, dtype=torch.int32), tensor(1040, dtype=torch.int32), tensor(6345, dtype=torch.int32), tensor(2488, dtype=torch.int32), tensor(3411, dtype=torch.int32), tensor(567, dtype=torch.int32), tensor(21426, dtype=torch.int32), tensor(482, dtype=torch.int32), tensor(9, dtype=torch.int32), tensor(1, dtype=torch.int32), tensor(2891, dtype=torch.int32), tensor(1558, dtype=torch.int32), tensor(356, dtype=torch.int32), tensor(75895, dtype=torch.int32), tensor(3482, dtype=torch.int32), tensor(742, dtype=torch.int32), tensor(2475, dtype=torch.int32), tensor(15853, dtype=torch.int32), tensor(264, dtype=torch.int32), tensor(7257, dtype=torch.int32), tensor(2, dtype=torch.int32), tensor(23, dtype=torch.int32), tensor(11070, dtype=torch.int32), tensor(88878, dtype=torch.int32), tensor(3994, dtype=torch.int32), ...]","[tensor(1641014864), tensor(1641154682), tensor(1641190269), tensor(1641362874), tensor(1641498144), tensor(1641537255), tensor(1641592469), tensor(1641701238), tensor(1641889088), tensor(1642568322), tensor(1642582444), tensor(1642773064), tensor(1642797822), tensor(1643876278), tensor(1643901015), tensor(1643970251), tensor(1644008163), tensor(1644320694), tensor(1644506198), tensor(1644585833), tensor(1644958620), tensor(1644996862), tensor(1645135398), tensor(1645387008), tensor(1645405467), tensor(1645488034), tensor(164553

In [8]:
df_targ_1.to_parquet('C:\\data_sber_com\\processed_train.pickle\\processed_train_10.parquet')

ArrowInvalid: ('Could not convert tensor([11584946,    14337,     5506,   107873,       51,    54826,    10713,\n           10022,     5026,        3,    76889,      456,     5489,     6965,\n             770,  6920812,     8319,    56051,     4178,    13929,     6343,\n           32819,     1225,      933,    13423,    17758,  5261698,      122,\n           30696,      882,     7234,      166,     5260,     3854,     1792,\n          906315,      120,    15511,    25802,     3419,      426,     1218,\n        13725419,     4887,  8960170,   155428,        0,     4424,     8789,\n             843,   137740,       92,     1027,      565,       92,   316354,\n            1777,        1,       24,    12852,     3799,      395,     5445,\n            8338,      290,    40652,    16612,   211773,       36,        3,\n            2873,    88664,    37692,       61,      133,      499,     1040,\n            6345,     2488,     3411,      567,    21426,      482,        9,\n               1, 

да, вычислительные мощности от Cloud.ru будут. инструкции есть в каналах по задачам или можете спрашивать у модераторов

- 1.5 гигов паркета заняло 15 гигов оперативы.
- 1.5 гигов паркета заняло 3.6 гигов оперативы.

In [6]:
def reduce_mem_usage(df, cat_cols=[]):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    for col in df.columns:
        if col in cat_cols:
            df[col] = df[col].astype("category")
    start_mem = df.memory_usage().sum() / 1024**2
    # print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if col_type.startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif col_type.startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    return df

In [16]:
#df_geo_1 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\geo_train.parquet')
#df_geo_1.head(2)
# Трейн геоданные вылетают по памяти при загрузке при 32 гигах оперативы

In [11]:
# directory = 'C:\data_sber_com\Hackathon\geo_train.parquet'
# df_geo = pd.DataFrame()
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     if os.path.isfile(f):
#         df_geo_iter = reduce_mem_usage(pd.read_parquet(f))
#         df_geo = pd.concat([df_geo_iter, df_geo])
#         df_geo.reset_index(drop = True, inplace=True)
#         del df_geo_iter
#         gc.collect()
#         #print('step')
#     #Скачиваю по шагам
# df_geo['geohash_4'] = df_geo['geohash_4'] - 30000
# df_geo['geohash_4'] = df_geo['geohash_4'].astype(np.int16)

по памяти загруженные трейн и тест занимают 21 Гб. Тест сразу загружать не обязательно

### Обработал таргеты

In [6]:
df_targ_1 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\train_target.parquet')
df_targ_1.head(2)

,mon,target_1,target_2,target_3,target_4,client_id
0,2022-06-30,0,0,0,0,1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e486174aaaa68ed2dbb572
1,2022-07-31,0,0,0,0,1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e486174aaaa68ed2dbb572


In [9]:
encoder = LabelEncoder()

In [11]:
df_targ_1['client_id_num'] = encoder.fit_transform(df_targ_1['client_id'])

In [16]:
integerMapping=dict(zip(df_targ_1['client_id'],df_targ_1['client_id_num']))

In [17]:
integerMapping

{'1d4ebf30ab5b981c60a6596e8daa4bdc0646163e50e486174aaaa68ed2dbb572': 97678,
 '1d55174bce3ef488233380aca206ca2fb51661b1c46495ebd42c18023fb9e121': 97769,
 '1d5d052f87d6bd22a30b5df160fffdcdd11014feffe75cd76e1f118d5f49f3a0': 97877,
 '1d68b588164639d64879b33da867818ea7401b6eb7caff004af9a03267634b5b': 98022,
 '1d817e82e1cc594e32e10217b716faa3ad079261975097da3e04ac51a96334a7': 98330,
 '1d87528929da4c0107c498bbac5bec8bf43d0b99440097f87b10344364dca706': 98412,
 '1d9df96a741dd1981f4cbc636f3ff24e46fd13f085d68760c1b45f45283c0c3a': 98690,
 '1daf5849cf68892983c584c8ee97b8329307584a834e795142be31f00e5900f9': 98917,
 '1db12b0b2d57f8d612c86279b0d89a8674d0ef2f2ad4b2949a72ac14c46fe106': 98945,
 '1db9bf462cb2e889d091df2455b61231f338c812de99597a0fc46f4c19cebb64': 99056,
 '1dcf9c37489219b8b801e403052a8800ce0114a8ca4d63f45e4b943ec5a835ed': 99346,
 '1deb21e0f461b5d48ea42922eed71e65dcaccc7506cda2c791f2e9051ffea947': 99713,
 '1e0025ba3d4554c8a8fce014afbbb6b578b9f5ffb2d34a998759945f4500ca9c': 99986,
 '1e140e1d13

In [44]:
# save it to a file
with open("C:\\data_sber_com\\dict_cl_id_to_num.pickle", "wb") as file:
    pickle.dump(integerMapping, file)
with open("C:\\data_sber_com\\encoder_cl_id.pickle", "wb") as file:
    pickle.dump(encoder, file)
# Сохранил словарь для кодирования
# Сохранил экземпляр кодировщика LabelEncoder() (уже  обученный)

In [20]:
integerMapping.get('0a5b204997551b0378eaa554082b45a82c821691eb2dfee709cb2ebc49479ca6')

In [21]:
set_targ_1 = set(df_targ_1['client_id'])

In [12]:
df_targ_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10246704 entries, 0 to 10246703
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   mon            object
 1   target_1       int32 
 2   target_2       int32 
 3   target_3       int32 
 4   target_4       int32 
 5   client_id      object
 6   client_id_num  int32 
dtypes: int32(5), object(2)
memory usage: 351.8+ MB


In [24]:
df_targ_2 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\test_target_b.parquet')
df_targ_2.head(2)

,mon,target_1,target_2,target_3,target_4,client_id
0,2022-05-31,0,0,0,0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671d0bcb425e90932db07
1,2022-05-31,0,0,0,0,0433d23e224b7a520656da6181efadb8d556bb293158c9304be4ce4c425bca01


In [25]:
set_targ_2 = set(df_targ_2['client_id'])

In [10]:
df_targ_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439362 entries, 0 to 1439361
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   mon        1439362 non-null  object
 1   target_1   1439362 non-null  int32 
 2   target_2   1439362 non-null  int32 
 3   target_3   1439362 non-null  int32 
 4   target_4   1439362 non-null  int32 
 5   client_id  1439362 non-null  object
dtypes: int32(4), object(2)
memory usage: 43.9+ MB


Размер меньше гига, извращаться смысла нет

### Обработал геоданные

In [10]:
#df_geo_2 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\geo_test.parquet')
#df_geo_2.head(2)
#df_geo_2['geohash_4'] = df_geo_2['geohash_4'] - 30000
#df_geo_2['geohash_4'] = df_geo_2['geohash_4'].astype(np.int16)
#pd.to_pickle(df_geo_2, 'C:\\data_sber_com\\geo_test.pickle')
df_geo_2 = pd.read_pickle('C:\\data_sber_com\\geo_test.pickle')
df_geo_2.info()
#Тестовые геоданные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139047888 entries, 0 to 139047887
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   client_id   object        
 1   event_time  datetime64[ns]
 2   geohash_4   int16         
 3   geohash_5   int32         
 4   geohash_6   int32         
dtypes: datetime64[ns](1), int16(1), int32(2), object(1)
memory usage: 3.4+ GB


In [18]:
#pd.to_pickle(df_geo, 'C:\\data_sber_com\\geo_train.pickle')
df_geo_1 = pd.read_pickle('C:\\data_sber_com\\geo_train.pickle')
df_geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528278383 entries, 0 to 528278382
Data columns (total 5 columns):
 #   Column      Dtype        
---  ------      -----        
 0   client_id   object       
 1   event_time  datetime64[s]
 2   geohash_4   int16        
 3   geohash_5   int32        
 4   geohash_6   int32        
dtypes: datetime64[s](1), int16(1), int32(2), object(1)
memory usage: 12.8+ GB


In [22]:
set_geo_1 = set(df_geo_1['client_id'])

In [23]:
len(set_targ_1), len(set_targ_1 & set_geo_1), len(set_geo_1)

(853892, 549171, 622304)

In [29]:
len(set_targ_1 - set_geo_1), len(set_geo_1 - set_targ_1)

(304721, 73133)

In [27]:
len(set_geo_1 & set_targ_2)

0

В геоданных нет примерно 300 тыс клиентов из 800 тыс на трейне. И есть лишние 73 тыс клиентов у которых нет таргета
удалю их нафиг

In [32]:
df_geo_1 = df_geo_1.loc[df_geo_1['client_id'].isin(set_targ_1), :]

In [36]:
df_geo_1['client_id_num'] = encoder.transform(df_geo_1['client_id'])

TypeError: transform() got an unexpected keyword argument 'inplace'

In [37]:
df_geo_1.drop(columns = ['client_id'], inplace= True)

In [39]:
df_geo_1.reset_index(drop = True, inplace=True)

In [40]:
df_geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468349475 entries, 0 to 468349474
Data columns (total 5 columns):
 #   Column         Dtype        
---  ------         -----        
 0   event_time     datetime64[s]
 1   geohash_4      int16        
 2   geohash_5      int32        
 3   geohash_6      int32        
 4   client_id_num  int32        
dtypes: datetime64[s](1), int16(1), int32(3)
memory usage: 9.6 GB


In [41]:
pd.to_pickle(df_geo_1, 'C:\\data_sber_com\\geo_train_small.pickle')

### Обработал диалоги

In [12]:
df_dial_1 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\dial_train.parquet')
df_dial_1.head(2)

,client_id,event_time,embedding
0,a039ad3b595d4f5b1a318076286b0e64e87e8dd9ab4795538e71ea081e4c320a,2022-05-02 12:24:30.565231,"[-0.003248451, 0.14023119, 0.05890738, 0.41109994, 0.07393208, 0.083305374, -0.118855335, -0.16710976, 0.051785704, -0.030376695, 0.1320928, 0.18384068, -0.07444467, 0.12789042, -0.09435633, 0.07923526, 0.052794553, -0.02205635, -0.036279026, 0.10017542, -0.9960928, 0.080878586, 0.017143002, 0.094622076, 0.16854551, -0.024534043, 0.08577774, -0.15141144, -0.117816925, 0.17342047, -0.38676298, -0.98543, -0.4961583, -0.18746477, -0.045532953, 0.030241277, -0.09392095, -0.033651214, -0.012200904, 0.061118733, 0.016686805, -0.07977645, 0.14723924, -0.38142955, 0.096580364, 0.16316125, 0.10029106, -0.11195236, 0.1849718, -0.045640968, 0.009499858, -0.061718464, 0.104066685, -0.2470839, -0.33932802, -0.06392263, -0.12982036, -0.100063264, -0.043193515, 0.3889805, -0.15532328, -0.123223014, -0.015988734, 0.05538417, 0.06252296, 0.02433936, -0.25206217, 0.13525571, -0.040885683, 0.08594197, 0.095362805, 0.027628299, 0.115431815, -0.16551313, -0.09306711, 0.04200701, -0.05200913, -0.030655589, -0.1417534, 0.05354739, 0.041690897, -0.17592362, 0.1939487, -0.35560033, 0.08654073, -0.15924326, -0.10781503, 0.18644665, -0.07377161, 0.28355986, -0.1668874, 0.013944658, 0.08476634, -0.034777846, -0.14249657, 0.011290711, 0.3791724, 0.123685956, 0.08190007, 0.023871835, ...]"
1,a039ad3b595d4f5b1a318076286b0e64e87e8dd9ab4795538e71ea081e4c320a,2022-10-28 09:05:18.394466,"[0.058927413, -0.0077230707, 0.23162016, 0.08985915, -0.04366154, 0.26320234, 0.083884165, 0.14597857, -0.19011363, 0.16441594, -0.06012006, -0.1701795, -0.1851537, -0.09783656, 0.10327868, -0.041035224, 0.4402991, 0.09374453, 0.060904086, -0.10873243, -0.99979854, -0.18244001, -0.12320468, -0.09490803, -0.1697146, 0.14569312, -0.092801996, 0.12635809, 0.12272047, 0.16386105, -0.17951626, -0.9989935, 0.0013097555, 0.26328367, 0.15804814, -0.017745242, 0.10213167, 0.11432658, 0.1555793, -0.1283121, -0.09273346, 0.04133641, -0.107426, 0.11411573, -0.1190452, -0.1034097, -0.0066782106, 0.05184966, -0.11600092, 0.048792575, -0.009015555, 0.057444148, 0.3313736, 0.03452754, 0.0880137, 0.076623425, 0.01175757, 0.17738822, 0.15053481, 0.037991278, 0.10793393, 0.11857061, 0.1786512, -0.11701684, -0.14841908, -0.16499576, -0.16561626, 0.035223015, 0.3598169, -0.07584453, -0.086752325, -0.18532515, -0.08296555, 0.081959255, 0.097841516, -0.1603877, 0.14361936, 0.0011633665, 0.067454755, -0.004370941, -0.15154216, -0.24743287, -0.088384554, 0.16356026, -0.07632007, 0.088878594, 0.1393312, -0.13301912, 0.17198834, 0.006452961, 0.030473644, 0.34871268, -0.13124374, 0.002873975, -0.24547297, -0.18907811, -0.51252025, -0.051872507, -0.12644912, -0.022756059, ...]"


In [13]:
df_dial_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215209 entries, 0 to 1215208
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   client_id   1215209 non-null  object        
 1   event_time  1215209 non-null  datetime64[ns]
 2   embedding   1215209 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 27.8+ MB


In [14]:
len(df_dial_1.loc[6, 'embedding'])

768

In [15]:
df_dial_2 = pd.read_parquet('C:\\data_sber_com\\Hackathon\\dial_test.parquet')
df_dial_2.head(2)

,client_id,event_time,embedding
0,08b3569cdfd015e4ef7ed3632cf86bcac5d0bd1ed7d1acb2b927422061bf75d5,2022-02-26 10:53:11.777539,"[0.11058904, -0.000544849, 0.29173028, 0.04335793, -0.08253054, 0.26471514, 0.11212362, 0.0017585218, -0.0434841, 0.13709532, -0.16348912, -0.014772451, -0.059018113, -0.16343145, 0.16430096, -0.058437485, 0.50995106, 0.11667214, 0.19587754, -0.18820532, -0.9999373, -0.0012848735, -0.03334645, -0.07376893, -0.19307537, 0.009831404, -0.111389115, 0.16032173, 0.12280922, -0.12481229, 0.1162764, -0.9998584, 0.08006747, 0.40236065, 0.11414312, 0.10286441, 0.17233877, 0.19434722, 0.13427119, -0.14477538, -0.19162615, 0.083006755, -0.13242798, 0.013626641, -0.13442205, -0.08674131, 0.18541184, 0.13181259, -0.06492309, 0.1348678, -0.13448137, 0.1846195, 0.35020477, 0.15718383, 0.1480078, 0.07655953, 0.11634133, 0.1781386, 0.1960089, 0.090251796, 0.16072239, 0.067615606, 0.078856386, -0.06843506, -0.056038536, -0.031324007, -0.12245976, 0.08787143, 0.23191491, -0.119031414, -0.15348889, -0.026247477, -0.11740036, 0.0820352, 0.18208872, -0.060840227, 0.19297723, 0.07071329, 0.08483942, -0.10275867, -0.013819134, -0.4659171, -0.06008183, 0.073420055, -0.05397873, 0.15139805, 0.0998945, -0.019863611, 0.004429525, 0.07047968, 0.058072735, 0.21737549, -0.13817166, 0.03097654, -0.35401464, -0.036288418, -0.77360326, -0.041032273, -0.019554978, -0.26465565, ...]"
1,08b3569cdfd015e4ef7ed3632cf86bcac5d0bd1ed7d1acb2b927422061bf75d5,2022-05-01 09:37:13.140523,"[0.0020903826, 0.07218458, 0.22001038, 0.030185219, -0.18575323, 0.37926808, 0.0038193495, 0.08939557, -0.10961272, 0.058035064, -0.015358212, -0.09299542, -0.37069184, -0.07607292, 0.26934466, -0.15668145, 0.42222443, 0.01371746, 0.028500278, -0.27321997, -0.999886, -0.085755154, 0.11685055, -0.19512479, -0.06840147, 0.30058867, -0.040237803, 0.03660727, 0.015730534, -0.013495508, -0.060389247, -0.9996053, 0.30438486, 0.28517947, 0.08494879, 0.013320592, 0.068358704, 0.26692122, 0.075880274, -0.040706787, -0.062876746, -0.019064423, -0.22389169, 0.05699866, -0.012390065, -0.0011480481, 0.10120439, -0.018821266, -0.027358357, 0.17505214, 0.07467893, 0.13475704, 0.2490472, -0.07791912, 0.031395767, 0.22910129, 0.14688168, 0.10146578, 0.30019918, -0.1639895, 0.092160806, 0.07789938, 0.10312028, -0.39498162, -0.3101427, -0.31882843, -0.011907396, 0.15779969, 0.2977931, -0.00844455, -0.024271656, -0.30865175, -0.01876453, 0.19525819, 0.06052651, -0.32920304, 0.26999763, 0.14362891, 0.19497603, -0.084451824, -0.29965866, -0.58536, -0.17760973, 0.13213478, 0.025203537, 0.029785443, 0.353774, -0.08618523, 0.24686055, -0.18059668, 0.019712005, 0.092205144, 0.16939005, 0.3426816, -0.22294234, -0.3425548, -0.5095644, -0.15285847, -0.0616511, -0.36823317, ...]"


## Быстрый бустинг по одному эмбедингу

In [27]:
df_targ_1['month'] = pd.to_datetime(df_targ_1['mon']).dt.month

In [28]:
df_targ_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10246704 entries, 0 to 10246703
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   mon        object
 1   target_1   int32 
 2   target_2   int32 
 3   target_3   int32 
 4   target_4   int32 
 5   client_id  object
 6   month      int32 
dtypes: int32(5), object(2)
memory usage: 351.8+ MB


In [33]:
df_targ_1.groupby('client_id')['target_4'].sum().sort_values()

client_id
000006265d27d1166ed67506682be7380007a5bead4362f0a9795f7d97fb08e3     0
a8c62a5e0db7e6af7b956f4d7a784fe7542435741d146fada9e33e0371743022     0
a8c639e6ef7d58d00fc4499a6b9feacae95933ac9ff05ffdf14b785a6c5bf23b     0
a8c640cb2f0543dfa82d994d34f7ff4ff9f2eafdd89e60434296f3519d84e659     0
a8c661c9f99cc24943c52687687f2fa498a961b2be983ef03c5da300ae5ff2fd     0
                                                                    ..
3b7b0919d71dbf3191d66714f917084ecf2d0a19940aa036207323813ce23c1e     8
17457426a68469ce4381003faa80c733cd028172f4e2fb4b94badc1d0d47cbaf     9
6d4f8d0b1ced9fb97c804c5b48a18cffc7ffa4b6d341377be6eafcd7de0b4ec0     9
49a7c458cd7ad297d9888747e21e93caee46879dfb9e2bdd57b87a36baad5158    10
c9edc911a086690a0375f69d7b251db1ed97bad025b3c7a53b75a1101ae7c242    10
Name: target_4, Length: 853892, dtype: int32

In [43]:
df_targ_1.sort_values(by = 'mon')

,mon,target_1,target_2,target_3,target_4,client_id,month
7093888,2022-02-28,0,0,0,0,861e879db5569fa4720e4c1a35aacba5bf70f36f61c30b48589a8d28a528890f,2
6778796,2022-02-28,0,0,0,0,1204f674f466b5b00d703491ab77ad91bfc6b6b53fcd6eb60689fdc8631aa293,2
9081524,2022-02-28,0,0,0,0,dc5e1e6e7a1cab5280af07c47af45fbbcd13b0e01279cf2cab4ca8584c17d0b2,2
728240,2022-02-28,0,0,0,0,f2749cf18006d13eccbd426faf911ea74ac321a5c1f292205b1f3455e641e007,2
3065720,2022-02-28,0,0,0,0,669348d578c78c69c7f2078bd9bb3f26dbbccb8fcfb88335b66984e2098672bd,2
...,...,...,...,...,...,...,...
1932955,2023-01-31,0,0,0,0,fa9fbcd9a24f2989350331f4ea8bbc5fa51160dc13a6449d579dad4cae190a43,1
9466787,2023-01-31,0,0,0,0,de858715d605aec6fbff55149f6af0cfa6e1d17dafe27aab200c22eaf7ac2d2f,1
1188919,2023-01-31,0,0,0,0,d8c9b13159cd422e7eb94a534074407560b98bdf7a035f7300d78407b2eb1644,1
4468991,2023-01-31,0,0,0,0,958beada3356957ae29828664456a7dac3499ccbc3d8168ab0d1c61fdf9a3db3,1


In [ ]:
'target_1' - 6 max
'target_2' - 9 max
'target_3' - 7 max
'target_4' - 10 max

In [36]:
df_dial_1['month'] = df_dial_1['event_time'].dt.month

In [42]:
df_targ_1['month'].unique()

array([ 6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5])

In [41]:
df_dial_1['month'].unique()

array([ 5, 10, 12,  9,  7,  3,  4,  6,  2, 11,  1,  8])

In [55]:
df_targ_1.loc[df_targ_1['month'] == 1, 'month'] == 13
df_dial_1['month'] = df_dial_1['event_time'].dt.month
df_dial_1.loc[df_dial_1['event_time'] <= '2022-01-31', 'month'] = 0
df_dial = df_dial['month'] + 1
df_dial = df_dial_1.sort_values(by = 'event_time').drop_duplicates(subset=['client_id', 'month'], keep='last') 
# 'first' -удаляет дубли КРОМЕ первого (остаются самые новые, добавление поздних данных не меняет ситуацию)
#['month']
# 'last'  -удаляет дубли КРОМЕ последнего (остаются самые старые)

In [56]:
# expand df.listcol into its own dataframe
tags = df_dial['embedding'].apply(pd.Series)

# rename each variable is listcol
tags = tags.rename(columns = lambda x : 'listcol_' + str(x))

# join the tags dataframe back to the original dataframe
df_dial = pd.concat([df_dial[['client_id', 'event_time', 'month']], tags[:]], axis=1)

In [57]:
df_dial.groupby('client_id')['month'].count().sort_values()

client_id
80362e0ef171151f4da725014f698191e2f64cb490fba072479212b2f0722508     1
87ad7d9dd8be2017b0c7c26b647cea06118e13a07d97bf6c99bd47656f76018f     1
87af1a3b4248c7eb8901d1156a9b211764e881b95c0fb791dcb9210a37f146a7     1
87af719a9be7388f1c0233f2f7ed6e9597258cc988fc9efaa362dc353e0f4cd8     1
87b0658eebcc5aa29322ef327dc17e746379706af1bfb0a2cf920bbaa927132c     1
                                                                    ..
9aa096cc3a7e67c76ae866ddce7e6d50dbfdc3030d5406b343aaeb87c32a2c1b    12
2326bf6cdc6081c54993d6995c0454f3c191d24c27bf05877ba4d258baf51414    12
88e16e0728c137b5c659bac3038c5dc84735cfa878b12d51c8ed6349ad1d7143    12
e548f4cd3f66217d71e464ceab89587d0d53d4a640a01bbb043aa770c79955a4    12
c1458f0836a9b90fdc81c57d7f83b8214130238fd419417aa620feb4968f261c    12
Name: month, Length: 326803, dtype: int64

Чтобы свести группировку и таргет в модель нужно добавить к месяцу группировки 1 а 
В таргете нужно заменить мес 1 на 13
чтобы они были на одной линии.

In [58]:
df_first_o = df_dial.merge(df_targ_1, on = ['client_id', 'month'])

In [59]:
df_first_o

client_id  \
0       21618442bd9695c76b39b88230e4a1a5864a3ec1072be0880104041ff8f2e857   
1       cf6ab6d86a266f78d60323af2cd0d820f33245fbf68f760d8ad59c80fba0b8ae   
2       027292ad82fce667872244357b88eb7ba83ac9ff26004da7db37fe37f21a80d1   
3       b6e2e12161916bbea7f03c665107fcdf8d62e722db8b3d081578c4b3c03e9337   
4       79f208ba9431f691662e3f0aef5b8bfd5e97890c291530f1a65033ebdfaefb1a   
...                                                                  ...   
580337  f83efd0cffc3754f6c820ee6cc89e752767422281b95b5404853e99264e94857   
580338  7709a2f4c10dec77d58dd08bcd1ee89e9526f586df0bd1833b41f17a92062ffa   
580339  9ec91c7c3113bfa464e9a4480cf75a5ee04568720d6c328879ac74d216f0b658   
580340  7820f787a19c0ece22416321858e2e7a50f618fc7f0fc2f7fb1657d8f217cfd3   
580341  ed62ed94f7db51bfae18d3985d6ef3d9d6343022bb63146b2676824019d08a4e   

                       event_time  month  listcol_0  listcol_1  listcol_2  \
0      2022-01-31 00:11:30.277715      1   0.155976  -0.109147   0.364597   
1      2022-01-31 00:14:13.073191      1  -0.056714   0.048507   0.128809   
2      2022-01-31 00:23:16.614171      1  -0.138885   0.007271   0.110688   
3      2022-01-31 00:43:58.441043      1   0.319802   0.153301   0.464430   
4      2022-01-31 00:44:00.444449      1   0.060182   0.039364   0.040189   
...                           ...    ...        ...        ...        ...   
580337 2022-12-30 20:10:00.589742     12   0.211503  -0.023011   0.568452   
580338 2022-12-30 20:11:27.240705     12   0.163536   0.172420   0.339328   
580339 2022-12-30 20:22:11.003983     12   0.336912  -0.212563   0.498243   
580340 2022-12-30 20:33:11.870394     12   0.595460  -0.265606   0.752883   
580341 2022-12-30 20:50:45.829493     12   0.592609  -0.240373   0.561132   

        listcol_3  listcol_4  listcol_5  listcol_6  listcol_7  listcol_8  \
0       -0.078250  -0.163684   0.331916   0.141452   0.258483  -0.284682   
1        0.435182   0.161484   0.106517  -0.117520  -0.002647   0.164294   
2        0.299489   0.128053   0.072068  -0.128023   0.000561   0.170046   
3       -0.019686  -0.292730   0.512278   0.303929   0.383026  -0.391550   
4        0.013057  -0.068343   0.301958  -0.183526   0.125592  -0.123046   
...           ...        ...        ...        ...        ...        ...   
580337  -0.274004  -0.317651   0.529500   0.338442   0.460257  -0.490408   
580338   0.150005  -0.105811   0.338358   0.130756   0.071822  -0.124213   
580339  -0.566399  -0.252314   0.478898   0.307954   0.433517  -0.430623   
580340  -0.433020  -0.329463   0.539627   0.348277   0.456226  -0.671576   
580341  -0.410243  -0.262347   0.496875   0.317479   0.421986  -0.486776   

        listcol_9  listcol_10  listcol_11  listcol_12  listcol_13  listcol_14  \
0        0.240193   -0.148067   -0.289243   -0.336036   -0.011462    0.049916   
1       -0.012647    0.061107    0.188613   -0.089695    0.000957   -0.027664   
2       -0.039977    0.085649   -0.184589   -0.082126    0.048467    0.165185   
3        0.217169   -0.298264   -0.204487   -0.462296   -0.371733    0.356236   
4        0.184890    0.132896   -0.153524   -0.044942   -0.137768    0.112913   
...           ...         ...         ...         ...         ...         ...   
580337   0.405658   -0.227543   -0.482298   -0.523766   -0.235461    0.445767   
580338   0.023233   -0.197321   -0.072580   -0.313679   -0.038781    0.019812   
580339   0.589592   -0.346297   -0.414781   -0.489414   -0.423883    0.583027   
580340   0.438215   -0.595216   -0.474923   -0.726808   -0.449993    0.449439   
580341   0.448163   -0.384705   -0.429049   -0.513453   -0.414735    0.429689   

        listcol_15  listcol_16  listcol_17  listcol_18  listcol_19  \
0        -0.148799    0.795789    0.029636    0.190828   -0.002452   
1         0.105297    0.119096   -0.049307   -0.049468    0.031618   
2         0.119936    0.300271    0.130969   -0.039852   -0.154825   
3        -0.097715    0.709056    0.139004    0.124464   -0

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df_first_o.drop(columns = ['client_id'
                                                                               , 'event_time', 'mon', 'target_1'
                                                                               , 'target_2', 'target_3', 'target_4']),
                                                    df_first_o['target_1'],
                                                    random_state=123, test_size = 0.3)

In [63]:
xgb_classifier = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic',
                                   tree_method='hist', eta=0.1, max_depth=3, random_state = 123, scale_pos_weight=8)
xgb_classifier.fit(X_train, y_train)
'roc_auc -', roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:, 1])

('roc_auc -', 0.6463776895129885)

In [64]:
feature_important = xgb_classifier.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

name_fit = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
name_fit

,score
month,89.0
listcol_615,40.0
listcol_678,20.0
listcol_262,17.0
listcol_735,16.0
...,...
listcol_420,1.0
listcol_426,1.0
listcol_431,1.0
listcol_434,1.0


In [65]:
X_train, X_test, y_train, y_test = train_test_split(df_first_o[list(name_fit.head(300).index)].drop(columns = ['month']),
                                                    df_first_o['target_1'],
                                                    random_state=123, test_size = 0.3)

xgb_classifier = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic',
                                   tree_method='hist', eta=0.1, max_depth=3, random_state = 123, scale_pos_weight=8)
xgb_classifier.fit(X_train, y_train)
'roc_auc -', roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:, 1])

('roc_auc -', 0.6230103019646347)

In [66]:
xgb_classifier_2 = xgb.XGBClassifier(subsample = 0.8,
                                     reg_lambda = 1.5,
                                     n_estimators = 350,
                                     min_child_weight = 20,
                                     max_depth = 4,
                                     max_delta_step = 0.9,
                                     learning_rate = 0.04,
                                     gamma = 3,
                                     colsample_bytree = 0.8,
                                     objective ='binary:logistic',
                                     tree_method='hist', eta=0.1, random_state=123, scale_pos_weight=2.1)
xgb_classifier_2.fit(X_train, y_train)
roc_auc_score(y_test, xgb_classifier_2.predict_proba(X_test)[:, 1])

0.6268240899798664

## Предварительный анализ

In [ ]:
df_4 = pd.read_parquet('data\\part-0 (4).parquet')
df_4.head(2)

In [ ]:
df_5 = pd.read_parquet('data\\part-0 (5).parquet')
df_5.head(2)

In [ ]:
df_6 = pd.read_parquet('data\\part-0 (6).parquet')
df_6.head(2)

In [ ]:
df_7 = pd.read_parquet('data\\part-0 (7).parquet')
df_7.head(2)

In [ ]:
df_5.head()

In [ ]:
df_4[['target_1', 'target_2', 'target_3', 'target_4']].sum()

In [ ]:
df_5[['target_1', 'target_2', 'target_3', 'target_4']].sum()

In [ ]:
df_5.groupby('mon')[['target_1', 'target_2', 'target_3', 'target_4']].sum().sort_index()

In [ ]:
df_5.info()